In [1]:
# MOUNTING THE DRIVE TO THE COLLAB NOTEBOOK
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
pubchem = pd.read_csv("/content/drive/MyDrive/clean_pubchem_data.csv")

In [3]:
combined_data = pd.read_csv("/content/drive/MyDrive/combined_data_AR_Bioassay.csv")

In [4]:
combined_data.head()

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Standard.Type,Standard.Relation,...,PubChem.Standard.Value,Activity.Comment,source,Data.Validity.Comment,IC50,Target.Accession.s.,Ligand,Target,Ki,Kd
0,1,103243193,10539813.0,C1=CC(=CC=C1CC2=CC(=O)C3=C(O2)C=CC(=C3)O)O,Unspecified,NaN,NaN,NaN,Activity,=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,103243498,10084377.0,C1=CC(=CC=C1CC2=CC(=O)C3=C(O2)C=C(C=C3)O)O,Unspecified,NaN,NaN,NaN,Activity,=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,103243498,10084377.0,C1=CC(=CC=C1CC2=CC(=O)C3=C(O2)C=C(C=C3)O)O,Unspecified,NaN,NaN,NaN,Activity,=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,103243498,10084377.0,C1=CC(=CC=C1CC2=CC(=O)C3=C(O2)C=C(C=C3)O)O,Unspecified,NaN,NaN,NaN,Activity,=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,103243193,10539813.0,C1=CC(=CC=C1CC2=CC(=O)C3=C(O2)C=CC(=C3)O)O,Unspecified,NaN,NaN,NaN,Activity,=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
num_rows, num_columns = combined_data.shape
print(num_rows, num_columns)

25904 22


In [6]:
combined_data.isnull().sum()

PUBCHEM_RESULT_TAG                   0
PUBCHEM_SID                          0
PUBCHEM_CID                        130
PUBCHEM_EXT_DATASOURCE_SMILES      130
PUBCHEM_ACTIVITY_OUTCOME             0
PUBCHEM_ACTIVITY_SCORE           25904
PUBCHEM_ACTIVITY_URL             25904
PUBCHEM_ASSAYDATA_COMMENT        19830
Standard.Type                        0
Standard.Relation                 6598
Standard.Value                    6600
Standard.Units                    6616
PubChem.Standard.Value           16804
Activity.Comment                 19846
source                            8110
Data.Validity.Comment            25062
IC50                             25470
Target.Accession.s.              25378
Ligand                           25378
Target                           25378
Ki                               25826
Kd                               25890
dtype: int64

In [7]:
combined_data.duplicated().sum()

13112

In [8]:
print(combined_data.columns)

Index(['PUBCHEM_RESULT_TAG', 'PUBCHEM_SID', 'PUBCHEM_CID',
       'PUBCHEM_EXT_DATASOURCE_SMILES', 'PUBCHEM_ACTIVITY_OUTCOME',
       'PUBCHEM_ACTIVITY_SCORE', 'PUBCHEM_ACTIVITY_URL',
       'PUBCHEM_ASSAYDATA_COMMENT', 'Standard.Type', 'Standard.Relation',
       'Standard.Value', 'Standard.Units', 'PubChem.Standard.Value',
       'Activity.Comment', 'source', 'Data.Validity.Comment', 'IC50',
       'Target.Accession.s.', 'Ligand', 'Target', 'Ki', 'Kd'],
      dtype='object')


# preprocessing

In [9]:
#dropping  columns
columns_to_remove = ['PUBCHEM_ACTIVITY_SCORE', 'PUBCHEM_ACTIVITY_URL', 'PUBCHEM_ASSAYDATA_COMMENT',
                     'Data.Validity.Comment', 'IC50', 'Target.Accession.s.', 'Ligand', 'Target', 'Ki', 'Kd']

combined_data = combined_data.drop(columns=columns_to_remove)

In [10]:
columns_to_remove = ['PubChem.Standard.Value', 'Activity.Comment', 'source']

combined_data = combined_data.drop(columns=columns_to_remove)

In [11]:
combined_data.isnull().sum()

PUBCHEM_RESULT_TAG                  0
PUBCHEM_SID                         0
PUBCHEM_CID                       130
PUBCHEM_EXT_DATASOURCE_SMILES     130
PUBCHEM_ACTIVITY_OUTCOME            0
Standard.Type                       0
Standard.Relation                6598
Standard.Value                   6600
Standard.Units                   6616
dtype: int64

In [12]:
# Check if the compounds in 'PUBCHEM_CID' of the new dataset are present in the 'cid' column of the previous dataset
same_compounds = combined_data['PUBCHEM_CID'].isin(pubchem['cid'])

# Count the number of compounds that are the same
num_same_compounds = same_compounds.sum()

print("Number of compounds with the same CID:", num_same_compounds)

Number of compounds with the same CID: 25774


In [13]:
#create a new df with informative columns only
new_df = combined_data[['PUBCHEM_CID', 'PUBCHEM_EXT_DATASOURCE_SMILES', 'PUBCHEM_ACTIVITY_OUTCOME']]

new_df.to_csv('new_dataframe.csv', index=False)

In [14]:
new_df = pd.read_csv('new_dataframe.csv')

# Merge with the 'pubchem' dataset based on 'PUBCHEM_CID'
merged_df = pd.merge(pubchem, new_df, left_on='cid', right_on='PUBCHEM_CID', how='inner')

# Drop the redundant 'PUBCHEM_CID' column
merged_df = merged_df.drop(columns=['PUBCHEM_CID'])

# Print the merged DataFrame
print(merged_df.head())

        cid                                           cmpdname     mw  \
0  10412732  6-Fluoro-2-[[5-(trifluoromethyl)-1,3-benzothia...  477.4   
1  10412732  6-Fluoro-2-[[5-(trifluoromethyl)-1,3-benzothia...  477.4   
2  10412732  6-Fluoro-2-[[5-(trifluoromethyl)-1,3-benzothia...  477.4   
3  10412732  6-Fluoro-2-[[5-(trifluoromethyl)-1,3-benzothia...  477.4   
4  10412732  6-Fluoro-2-[[5-(trifluoromethyl)-1,3-benzothia...  477.4   

              mf  polararea  complexity  xlogp  heavycnt  hbonddonor  \
0  C21H11F4N3O4S      125.0       901.0    2.7        33           1   
1  C21H11F4N3O4S      125.0       901.0    2.7        33           1   
2  C21H11F4N3O4S      125.0       901.0    2.7        33           1   
3  C21H11F4N3O4S      125.0       901.0    2.7        33           1   
4  C21H11F4N3O4S      125.0       901.0    2.7        33           1   

   hbondacc  ...  gpfamilycnt neighbortype  \
0        10  ...            2        2D+3D   
1        10  ...            2       

In [15]:
merged_df.isnull().sum()

cid                               0
cmpdname                          0
mw                                0
mf                                0
polararea                         0
complexity                        0
xlogp                             0
heavycnt                          0
hbonddonor                        0
hbondacc                          0
rotbonds                          0
inchi                             0
isosmiles                         0
canonicalsmiles                   0
inchikey                          0
iupacname                         0
exactmass                         0
monoisotopicmass                  0
charge                            0
covalentunitcnt                   0
isotopeatomcnt                    0
totalatomstereocnt                0
definedatomstereocnt              0
undefinedatomstereocnt            0
totalbondstereocnt                0
definedbondstereocnt              0
undefinedbondstereocnt            0
pclidcnt                    

In [16]:
merged_df.head()

,cid,cmpdname,mw,mf,polararea,complexity,xlogp,heavycnt,hbonddonor,hbondacc,...,gpfamilycnt,neighbortype,annothits,annothitcnt,aids,cidcdate,sidsrcname,depcatg,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME
0,10412732,"6-Fluoro-2-[[5-(trifluoromethyl)-1,3-benzothia...",477.4,C21H11F4N3O4S,125.0,901.0,2.7,33,1,10,...,2,2D+3D,Biological Test Results|Classification|Literat...,4,19449|34201|34205|185756|230202|230203|230204|...,20061025,BindingDB|ChEMBL|ChemSpider|DiscoveryGate|IBM|...,Curation Efforts|Governmental Organizations|Jo...,C1C(=O)NC(=O)C12C3=C(C=CC(=C3)F)C(=O)N(C2=O)CC...,Unspecified
1,10412732,"6-Fluoro-2-[[5-(trifluoromethyl)-1,3-benzothia...",477.4,C21H11F4N3O4S,125.0,901.0,2.7,33,1,10,...,2,2D+3D,Biological Test Results|Classification|Literat...,4,19449|34201|34205|185756|230202|230203|230204|...,20061025,BindingDB|ChEMBL|ChemSpider|DiscoveryGate|IBM|...,Curation Efforts|Governmental Organizations|Jo...,C1C(=O)NC(=O)C12C3=C(C=CC(=C3)F)C(=O)N(C2=O)CC...,Unspecified
2,10412732,"6-Fluoro-2-[[5-(trifluoromethyl)-1,3-benzothia...",477.4,C21H11F4N3O4S,125.0,901.0,2.7,33,1,10,...,2,2D+3D,Biological Test Results|Classification|Literat...,4,19449|34201|34205|185756|230202|230203|230204|...,20061025,BindingDB|ChEMBL|ChemSpider|DiscoveryGate|IBM|...,Curation Efforts|Governmental Organizations|Jo...,C1C(=O)NC(=O)C12C3=C(C=CC(=C3)F)C(=O)N(C2=O)CC...,Unspecified
3,10412732,"6-Fluoro-2-[[5-(trifluoromethyl)-1,3-benzothia...",477.4,C21H11F4N3O4S,125.0,901.0,2.7,33,1,10,...,2,2D+3D,Biological Test Results|Classification|Literat...,4,19449|34201|34205|185756|230202|230203|230204|...,20061025,BindingDB|ChEMBL|ChemSpider|DiscoveryGate|IBM|...,Curation Efforts|Governmental Organizations|Jo...,C1C(=O)NC(=O)C12C3=C(C=CC(=C3)F)C(=O)N(C2=O)CC...,Unspecified
4,10412732,"6-Fluoro-2-[[5-(trifluoromethyl)-1,3-benzothia...",477.4,C21H11F4N3O4S,125.0,901.0,2.7,33,1,10,...,2,2D+3D,Biological Test Results|Classification|Literat...,4,19449|34201|34205|185756|230202|230203|230204|...,20061025,BindingDB|ChEMBL|ChemSpider|DiscoveryGate|IBM|...,Curation Efforts|Governmental Organizations|Jo...,C1C(=O)NC(=O)C12C3=C(C=CC(=C3)F)C(=O)N(C2=O)CC...,Unspecified


In [17]:
columns_to_drop = ['neighbortype', 'annothits', 'sidsrcname', 'depcatg']
merged_df = merged_df.drop(columns=columns_to_drop)


# feature engineering

In [18]:
print(merged_df.columns)

Index(['cid', 'cmpdname', 'mw', 'mf', 'polararea', 'complexity', 'xlogp',
       'heavycnt', 'hbonddonor', 'hbondacc', 'rotbonds', 'inchi', 'isosmiles',
       'canonicalsmiles', 'inchikey', 'iupacname', 'exactmass',
       'monoisotopicmass', 'charge', 'covalentunitcnt', 'isotopeatomcnt',
       'totalatomstereocnt', 'definedatomstereocnt', 'undefinedatomstereocnt',
       'totalbondstereocnt', 'definedbondstereocnt', 'undefinedbondstereocnt',
       'pclidcnt', 'gpidcnt', 'gpfamilycnt', 'annothitcnt', 'aids', 'cidcdate',
       'PUBCHEM_EXT_DATASOURCE_SMILES', 'PUBCHEM_ACTIVITY_OUTCOME'],
      dtype='object')


In [19]:
# Check the data types of columns in the merged DataFrame
print(merged_df.dtypes)

cid                                int64
cmpdname                          object
mw                               float64
mf                                object
polararea                        float64
complexity                       float64
xlogp                            float64
heavycnt                           int64
hbonddonor                         int64
hbondacc                           int64
rotbonds                           int64
inchi                             object
isosmiles                         object
canonicalsmiles                   object
inchikey                          object
iupacname                         object
exactmass                        float64
monoisotopicmass                 float64
charge                             int64
covalentunitcnt                    int64
isotopeatomcnt                     int64
totalatomstereocnt                 int64
definedatomstereocnt               int64
undefinedatomstereocnt             int64
totalbondstereoc

In [20]:
categorical_columns = ['cmpdname', 'mf', 'inchi', 'isosmiles', 'canonicalsmiles',
                       'inchikey', 'iupacname', 'aids', 'PUBCHEM_EXT_DATASOURCE_SMILES',
                       'PUBCHEM_ACTIVITY_OUTCOME']

# Display unique values and their counts for each categorical column
for column in categorical_columns:
    print("Column:", column)
    print("Number of unique values:", merged_df[column].nunique())
    print("Unique values:")
    print(merged_df[column].value_counts())
    print("\n")

Column: cmpdname
Number of unique values: 4301
Unique values:
Tolrestat                                                               380
Sorbinil                                                                338
Epalrestat                                                              262
Zopolrestat                                                             128
Trolox                                                                  122
                                                                       ... 
3-phenylspiro[1H-quinazoline-4,5'-imidazolidine]-2,2',4'-trione           2
2-amino-3'-methylspiro[1,3-thiazole-5,4'-1H-quinazoline]-2',4-dione       2
(4S)-3-methylspiro[1H-quinazoline-4,5'-imidazolidine]-2,2',4'-trione      2
2-amino-3'-methylspiro[1H-imidazole-4,4'-1H-quinazoline]-2',5-dione       2
Diclofenac sodium                                                         2
Name: cmpdname, Length: 4301, dtype: int64


Column: mf
Number of unique values: 3117
Unique values:
C

In [21]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Columns for one-hot encoding (large number of unique values)
one_hot_columns = ['cmpdname', 'inchi', 'isosmiles', 'iupacname', 'aids', 'PUBCHEM_EXT_DATASOURCE_SMILES']

# Columns for label encoding (smaller number of unique values)
label_columns = ['mf']

# Initialize encoders
one_hot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
label_encoder = LabelEncoder()

# One-hot encoding
one_hot_encoded = one_hot_encoder.fit_transform(merged_df[one_hot_columns])
one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=[f"{col}_{val}" for col in one_hot_columns for val in merged_df[col].unique()])

# Label encoding
label_encoded_df = merged_df[label_columns].apply(label_encoder.fit_transform)

# Concatenate the encoded columns with the original DataFrame
merged_df_encoded = pd.concat([merged_df.drop(columns=one_hot_columns + label_columns), one_hot_encoded_df, label_encoded_df], axis=1)

# Display the updated DataFrame
print(merged_df_encoded)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


             cid      mw  polararea  complexity     xlogp  heavycnt  \
0       10412732  477.40      125.0       901.0  2.700000        33   
1       10412732  477.40      125.0       901.0  2.700000        33   
2       10412732  477.40      125.0       901.0  2.700000        33   
3       10412732  477.40      125.0       901.0  2.700000        33   
4       10412732  477.40      125.0       901.0  2.700000        33   
...          ...     ...        ...         ...       ...       ...   
25769  156016680  457.00       97.9       629.0  3.600000        32   
25770      61253  282.33       87.0       269.0  3.700000        20   
25771      61253  282.33       87.0       269.0  3.700000        20   
25772    5018304  318.10       52.2       310.0  2.498303        20   
25773    5018304  318.10       52.2       310.0  2.498303        20   

       hbonddonor  hbondacc  rotbonds  \
0               1        10         2   
1               1        10         2   
2               1       

molecular descriptors

In [22]:
! pip install padelpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 44.8 MB/s eta 0:00:00


In [26]:
! unzip /content/drive/MyDrive/aroma/fingerprints_xml.zip

unzip:  cannot find or open /content/drive/MyDrive/aroma/fingerprints_xml.zip, /content/drive/MyDrive/aroma/fingerprints_xml.zip.zip or /content/drive/MyDrive/aroma/fingerprints_xml.zip.ZIP.


In [23]:
print(merged_df.columns)

Index(['cid', 'cmpdname', 'mw', 'mf', 'polararea', 'complexity', 'xlogp',
       'heavycnt', 'hbonddonor', 'hbondacc', 'rotbonds', 'inchi', 'isosmiles',
       'canonicalsmiles', 'inchikey', 'iupacname', 'exactmass',
       'monoisotopicmass', 'charge', 'covalentunitcnt', 'isotopeatomcnt',
       'totalatomstereocnt', 'definedatomstereocnt', 'undefinedatomstereocnt',
       'totalbondstereocnt', 'definedbondstereocnt', 'undefinedbondstereocnt',
       'pclidcnt', 'gpidcnt', 'gpfamilycnt', 'annothitcnt', 'aids', 'cidcdate',
       'PUBCHEM_EXT_DATASOURCE_SMILES', 'PUBCHEM_ACTIVITY_OUTCOME'],
      dtype='object')


Molecular Descriptors

# MODEL TRAINING


In [25]:
relevant_columns = ['cid', 'MolecularWeight', 'LogP', 'ecfp4_fingerprint', 'PUBCHEM_ACTIVITY_OUTCOME']

# Extract relevant columns
relevant_data = merged_df[relevant_columns]

# Display the extracted data
print(relevant_data.head())

KeyError: "['MolecularWeight', 'LogP', 'ecfp4_fingerprint'] not in index"

In [ ]:
# Define features (X) and target variable (y)
X = merged_df[['ecfp4_fingerprint', 'MolecularWeight', 'LogP', 'mw', 'polararea', 'complexity', 'xlogp', 'heavycnt', 'hbonddonor', 'hbondacc',
    'rotbonds', 'exactmass', 'monoisotopicmass', 'charge', 'covalentunitcnt',
    'isotopeatomcnt', 'totalatomstereocnt', 'definedatomstereocnt', 'undefinedatomstereocnt',
    'totalbondstereocnt', 'definedbondstereocnt', 'undefinedbondstereocnt']]  # Include all relevant features
y = merged_df['PUBCHEM_ACTIVITY_OUTCOME']

# Split the data into training, validation, and test sets
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Check the shapes of the datasets
print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)
print("Test set shape:", X_test.shape, y_test.shape)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

from rdkit import DataStructs

# Convert ExplicitBitVect objects to numpy arrays
X_train_ecfp4 = np.array([DataStructs.ConvertToNumpyArray(fp) for fp in X_train['ecfp4_fingerprint']])
X_test_ecfp4 = np.array([DataStructs.ConvertToNumpyArray(fp) for fp in X_test['ecfp4_fingerprint']])

# Train the classifier on the training set
svm_classifier.fit(X_train_ecfp4, y_train)

# Predictions on the test set
y_pred_test = svm_classifier.predict(X_test_ecfp4)





# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test, average='weighted')
recall = recall_score(y_test, y_pred_test, average='weighted')
f1 = f1_score(y_test, y_pred_test, average='weighted')
roc_auc = roc_auc_score(y_test, y_pred_test)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("ROC AUC score:", roc_auc)